# Spark DataFrame - Dates and Timestamps

**Imports and read csv file**

In [27]:
import findspark
findspark.init('/home/sedat/spark-3.3.2-bin-hadoop3')
from pyspark.sql import SparkSession
from pyspark.sql.functions import (dayofmonth, dayofyear, 
                                   weekofyear, format_number,
                                   hour, month, year, date_format, days)

In [3]:
spark = SparkSession.builder.appName('dates').getOrCreate()

In [5]:
df = spark.read.csv('appl_stock.csv', header=True, inferSchema=True)

In [6]:
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [11]:
df.head(1)

[Row(Date=datetime.datetime(2010, 1, 4, 0, 0), Open=213.429998, High=214.499996, Low=212.38000099999996, Close=214.009998, Volume=123432400, Adj Close=27.727039)]

**Show first 2 columns**

In [12]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

In [15]:
df.select(df.columns[0:2]).show()

+-------------------+------------------+
|               Date|              Open|
+-------------------+------------------+
|2010-01-04 00:00:00|        213.429998|
|2010-01-05 00:00:00|        214.599998|
|2010-01-06 00:00:00|        214.379993|
|2010-01-07 00:00:00|            211.75|
|2010-01-08 00:00:00|        210.299994|
|2010-01-11 00:00:00|212.79999700000002|
|2010-01-12 00:00:00|209.18999499999998|
|2010-01-13 00:00:00|        207.870005|
|2010-01-14 00:00:00|210.11000299999998|
|2010-01-15 00:00:00|210.92999500000002|
|2010-01-19 00:00:00|        208.330002|
|2010-01-20 00:00:00|        214.910006|
|2010-01-21 00:00:00|        212.079994|
|2010-01-22 00:00:00|206.78000600000001|
|2010-01-25 00:00:00|202.51000200000001|
|2010-01-26 00:00:00|205.95000100000001|
|2010-01-27 00:00:00|        206.849995|
|2010-01-28 00:00:00|        204.930004|
|2010-01-29 00:00:00|        201.079996|
|2010-02-01 00:00:00|192.36999699999998|
+-------------------+------------------+
only showing top

**Day of month, week of year and day of year**

In [26]:
df.select(df['Date'], dayofmonth(df['Date']), weekofyear(df['Date']), dayofyear(df['Date'])).show()

+-------------------+----------------+----------------+---------------+
|               Date|dayofmonth(Date)|weekofyear(Date)|dayofyear(Date)|
+-------------------+----------------+----------------+---------------+
|2010-01-04 00:00:00|               4|               1|              4|
|2010-01-05 00:00:00|               5|               1|              5|
|2010-01-06 00:00:00|               6|               1|              6|
|2010-01-07 00:00:00|               7|               1|              7|
|2010-01-08 00:00:00|               8|               1|              8|
|2010-01-11 00:00:00|              11|               2|             11|
|2010-01-12 00:00:00|              12|               2|             12|
|2010-01-13 00:00:00|              13|               2|             13|
|2010-01-14 00:00:00|              14|               2|             14|
|2010-01-15 00:00:00|              15|               2|             15|
|2010-01-19 00:00:00|              19|               3|         

**Year, month, hour**

In [37]:
df.select(df['Date'], year(df['Date']), month(df['Date']), hour(df['Date'])).show()

+-------------------+----------+-----------+----------+
|               Date|year(Date)|month(Date)|hour(Date)|
+-------------------+----------+-----------+----------+
|2010-01-04 00:00:00|      2010|          1|         0|
|2010-01-05 00:00:00|      2010|          1|         0|
|2010-01-06 00:00:00|      2010|          1|         0|
|2010-01-07 00:00:00|      2010|          1|         0|
|2010-01-08 00:00:00|      2010|          1|         0|
|2010-01-11 00:00:00|      2010|          1|         0|
|2010-01-12 00:00:00|      2010|          1|         0|
|2010-01-13 00:00:00|      2010|          1|         0|
|2010-01-14 00:00:00|      2010|          1|         0|
|2010-01-15 00:00:00|      2010|          1|         0|
|2010-01-19 00:00:00|      2010|          1|         0|
|2010-01-20 00:00:00|      2010|          1|         0|
|2010-01-21 00:00:00|      2010|          1|         0|
|2010-01-22 00:00:00|      2010|          1|         0|
|2010-01-25 00:00:00|      2010|          1|    

**Add year as a new column to df**

In [58]:
new_df = df.withColumn('Year', year(df['Date']))

In [72]:
results = new_df.groupBy('Year').mean().select(['Year', 'avg(Close)'])

In [74]:
new = results.withColumnRenamed('avg(Close)', 'Average Closing Price')
new.show()

+----+---------------------+
|Year|Average Closing Price|
+----+---------------------+
|2015|   120.03999980555547|
|2013|    472.6348802857143|
|2014|    295.4023416507935|
|2012|    576.0497195640002|
|2016|   104.60400786904763|
|2010|    259.8424600000002|
|2011|   364.00432532142867|
+----+---------------------+



In [78]:
new.select('Year', format_number('Average Closing Price', 2).alias('Average Closing Price')).show()

+----+---------------------+
|Year|Average Closing Price|
+----+---------------------+
|2015|               120.04|
|2013|               472.63|
|2014|               295.40|
|2012|               576.05|
|2016|               104.60|
|2010|               259.84|
|2011|               364.00|
+----+---------------------+

